<a href="https://colab.research.google.com/github/michalis0/DataScience_and_MachineLearning/blob/master/Assignements/Part%204/Assignment_part_four.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DSML investigation:

You are part of the Suisse Impossible Mission Force, or SIMF for short. You need to uncover a rogue agent that is trying to steal sensitive information.

Your mission, should you choose to accept it, is to find that agent before stealing any classified information. Good luck!

# Assignement part four
### Due 22.10
#### Identifying the suspects credit score
We received informations that the rogue agent has a good credit score.

Our spies at SIMF have managed to collect financial information relating to our suspects as well as a training dataset.

Create a Neural Network over the training dataset `df` to identify which of the suspects have a good Credit_Mix


## Getting to know our data

* Age: a users age
* Occupation: a users employment field
* Annual_Income: a users annual income
* Monthly_Inh_Salary: the calculated salary received by a given user on a monthly basis
* Num_Bank_Accounts: the number of bank accounts possessed by a given user
* Num_Credit_Cards: the number of credit card given user possesses
* Interest_Rate: The interest rate on those cards (if multiple then its the average)
* Num_of_Loans: The number of loans of each user
* Delay_from_due_date: payment tardiness of user
* Num_of_Delayed_Payment: the count of delayed payments
* Changed_Credit_Limit: NaN
* Num_Credit_Inquiries: NaN
* Credit_Mix: The users credit score
* Outsting_Debt: Outstanding debt
* Credit_Utilization_Ratio: the percentage of borrowed money over borrowing allowance
* Payment_of_Min_Amount: does the user usually pay the minimal amount (categorical)
* Total_EMI_per_month: Monthly repayments to be made
* Amount_invested_monthly: The amout put in an investment fun by the user on a monthly basis
* Payment_Behaviour: the users payment behavior (categorical)
* Monthly_Balance: The users end of the month balance
* AutoLoan: If the user has an active loan for their vehicule
* Credit-BuilderLoan: If the user has a loan to increase their credit score
* DebtConsolidationLoan, HomeEquityLoan, MortgageLoan, NotSpecified, PaydayLoan, PersonalLoan, StudentLoan: different types of loans(categorical features)



In [1]:
# Import required packages
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/master/Assignements/Part%204/data/train_classification.csv", index_col='Unnamed: 0').dropna()
suspects = pd.read_csv("https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/master/Assignements/Part%204/data/suspects.csv", index_col='Unnamed: 0').dropna()

In [3]:
df.head()

,Age,Occupation,Annual_Income,Monthly_Inh_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,...,Monthly_Balance,AutoLoan,Credit-BuilderLoan,DebtConsolidationLoan,HomeEquityLoan,MortgageLoan,NotSpecified,PaydayLoan,PersonalLoan,StudentLoan
0,23,Scientist,19114.12,1824.843333,3,4,3,4,3,7,...,186.266702,1,1,0,1,0,0,0,1,0
1,24,Scientist,19114.12,1824.843333,3,4,3,4,3,9,...,361.444004,1,1,0,1,0,0,0,1,0
3,24,Scientist,19114.12,4182.004291,3,4,3,4,4,5,...,343.826873,1,1,0,1,0,0,0,1,0
5,28,Teacher,34847.84,3037.986667,2,4,6,1,3,3,...,303.355083,0,1,0,0,0,0,0,0,0
8,35,Engineer,143162.64,4182.004291,1,5,8,3,8,1942,...,854.226027,2,0,0,0,0,1,0,0,0


In [4]:
df["Credit_Mix"].value_counts()

Credit_Mix
Standard    13421
Good         8963
Bad          6839
Name: count, dtype: int64

# 1. Preparing the data
## 1.1 Data cleaning
 Perform OHE over the "Occupation" feature

 Then, perform LE over Payment_of_Min_Amount and Payment_Behaviour

 _hint: As we will be testing only one model no need to define a pipeline_

In [28]:

# Your code here:
ohe = OneHotEncoder(sparse= False)
df_fitted = ohe.fit_transform(df[['Occupation']])
categories = ohe.get_feature_names_out(input_features=['Occupation'])
df_encoded = pd.DataFrame(df_fitted, columns=categories, index= df.index)
# Replace the 'Occupation' column with the one-hot encoded columns
df_encoded = pd.concat([df.drop(['Occupation'], axis=1), df_encoded], axis=1)

le = LabelEncoder()
df_encoded['Payment_of_Min_Amount'] = le.fit_transform(df_encoded['Payment_of_Min_Amount'])
df_encoded['Payment_Behaviour'] = le.fit_transform(df_encoded['Payment_Behaviour'])

print(df_encoded)






       Age  Annual_Income  Monthly_Inh_Salary  Num_Bank_Accounts  \
0       23       19114.12         1824.843333                  3   
1       24       19114.12         1824.843333                  3   
3       24       19114.12         4182.004291                  3   
5       28       34847.84         3037.986667                  2   
8       35      143162.64         4182.004291                  1   
...    ...            ...                 ...                ...   
49990   50       37188.10         3097.008333                  1   
49992   29       20002.88         1929.906667                 10   
49993   29       20002.88         1929.906667                 10   
49997   25       39628.99         3359.415833                  4   
49998   25       39628.99         4182.004291                  4   

       Num_Credit_Card  Interest_Rate  Num_of_Loan  Delay_from_due_date  \
0                    4              3            4                    3   
1                    4           

/home/nathan/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## 1.2 Dataset splitting

Split the dataset in two, first X with your independent features and then y with the dependent feature **CreditMix**.

Then perform :
* OneHotEncoding over the **CreditMix** feature.
* A MinMaxScaller over the independent features

In [27]:
# Your code here:
X = pd.DataFrame(df_encoded.drop(columns=['Credit_Mix']))
y = pd.DataFrame(df_encoded['Credit_Mix'])
#print(y.index)
y_fitted = ohe.fit_transform(y[['Credit_Mix']])
categories_credit_mix = ohe.get_feature_names_out(input_features=['Credit_Mix'])
y = pd.DataFrame(y_fitted, columns=categories_credit_mix, index= y.index)
# Replace the 'Occupation' column with the one-hot encoded columns
#y = pd.concat([y.drop(['Credit_mix'], axis=1), y], axis=1)
print(y)

#Define the scaler
scaler = MinMaxScaler()
#Fit the scaler
scaler.fit(X)
print(X)

       Credit_Mix_Bad  Credit_Mix_Good  Credit_Mix_Standard
0                 0.0              1.0                  0.0
1                 0.0              1.0                  0.0
3                 0.0              1.0                  0.0
5                 0.0              1.0                  0.0
8                 0.0              1.0                  0.0
...               ...              ...                  ...
49990             0.0              1.0                  0.0
49992             1.0              0.0                  0.0
49993             1.0              0.0                  0.0
49997             0.0              1.0                  0.0
49998             0.0              1.0                  0.0

[29223 rows x 3 columns]
       Age  Annual_Income  Monthly_Inh_Salary  Num_Bank_Accounts  \
0       23       19114.12         1824.843333                  3   
1       24       19114.12         1824.843333                  3   
3       24       19114.12         4182.004291     

/home/nathan/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


### 1.2.1 Train Test splitting
Now split the data in X_train, X_test, y_train, y_test, 

You can use test_size = 0.2 and a random_state of 42

In [31]:
# Your code here
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(23378, 42) (5845, 42) (23378, 3) (5845, 3)


### 1.2.2 final touches
Convert your datasets to `Torch tensors` of type `torch.float`

In [32]:
#Your code here:
X_train = torch.tensor(X_train.values, dtype=torch.float)
y_train = torch.tensor(y_train.values, dtype=torch.float)
X_test = torch.tensor(X_test.values, dtype=torch.float)
y_test = torch.tensor(y_test.values, dtype=torch.float)


# 2 Model preparation:

## 2.1 Define a Neural network model and instantiate it.
You can set the number of neurons to 150.

In [ ]:
# Define a neural network class here:


In [ ]:
# Instantiate your model here

## 2.2 finding the best model:
Identify, amongst the following options the best parameters for your model:

* `criterion` : [CrossEntropyLoss](https://anvilproject.org/guides/content/creating-links), [BCEWithLogitsLoss](hhttps://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)
* `iterations` : 150, 250, 500
* `learning rate` : 0.00005, 0.001, 12.031


_Hint: restart your runtime between each execution to ensure that previous neural networks dont interfere with your current one_

_You can evaluate your model based on it's accuracy over the test set_

In [ ]:
# Define your loss function here:

# Define your SGD optimizer for finding the weights of the network here


In [ ]:
# Perform your iterations here

## 2.3 Model Accuracy
Identify the models accuracy over the train and test parts of the training dataset

In [ ]:
# deactivate dropout layers

# Training accuracy


# Test accuracy


# 3. Predictions over the suspects dataset
## 3.1 Retrain a new model over the full training dataset
#### Please use the following parameters for this section:
* ``neurons`` = 150
* ``learning`` rate = 0.00005
* ``criterion`` = CrossEntropyLoss
* `iterations` = 500

_hint you may have to redo some preprocessing as you did in part one_

In [ ]:
# Define a new model here:

In [ ]:
# Define your MSE loss here:

# Define your SGD optimizer for finding the weights of the network here:


In [ ]:
# perform your training here

## 3.2 Predict over the suspects dataset

In [ ]:
# Predict which users have a good credit score here:
